# Using ICESat-2 ATL14/15, Gridded Arctic Land Ice Height to investigate ice-surface height anomalies

```{admonition} Learning Objectives
:class: tip
**Key learning outcomes:**
- 

**Practical skills that will lead to the learning outcomes:**
- 
```

## Computing environment

We'll be using the following open-source Python libraries in this notebook:

In [ ]:
# import internal libraries
import os

# import external libraries
import geopandas as gpd
import h5py
import icepyx as ipx
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import pandas as pd
import rioxarray
import s3fs
import xarray as xr

# In this tutorial, we will focus on southwest greenland, where active subglacial lakes have been detected

:::{figure-md} n-hemisphere-lakes
<img src="Livingstone2020Fig3a.png" alt="2022 inventory of subglacial lakes" class="bg-primary mb-1" width="600px">

Northern hemisphere subglacial lakes compiled in a global inventory by Livingstone and others (2020).
:::

```{figure} ../Livingstone2020Fig3a.png
---
height: 150px
name: directive-fig
---
Here is my figure caption!
```

We can investigate active subglacial lakes, which drain and fill episodically, use ice surface height anomalies as the overlying ice deforms when active lakes drain and fill. 

## Locate the Greenland active subglacial lakes
Consulting the supplementary data of the [Livingstone and others (2020) inventory](https://www.nature.com/articles/s43017-021-00246-9#Sec16), we can construct a [geopandas geodataframe](https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html) to investigate Greenland's subglacial.import_rows = np.arange(0,65)

In [ ]:
# Read in spreadsheet using pandas read_excel
url = 'https://static-content.springer.com/esm/art%3A10.1038%2Fs43017-021-00246-9/MediaObjects/43017_2021_246_MOESM1_ESM.xlsx'
use_cols = ['Name / Location', 'Lat. oN', 'Lon. oE', 'Lake Type', 'References']
import_rows = np.arange(0,65)
df = pd.read_excel(url, sheet_name='Greenland', usecols=use_cols, skiprows = lambda x: x not in import_rows)

# View pandas dataset head (first 5 rows)
df.head()

Alternatively, if we are working with a dataset that is not cloud hosted or available from a URL, we can upload the data to CryoCloud. 

First, create data_directory folder in your CryoCloud's base directory to stay organized.

Second, download the paper's supplementary data by [clicking here](https://static-content.springer.com/esm/art%3A10.1038%2Fs43017-021-00246-9/MediaObjects/43017_2021_246_MOESM1_ESM.xlsx).

Third, upload the supplementary data spreadsheet into your new data directory folder

In [ ]:
# Read in spreadsheet using pandas read_excel
use_cols = ['Name / Location', 'Lat. oN', 'Lon. oE', 'Lake Type', 'References']
import_rows = np.arange(0,65)
# You may add a path/to/file if you'd like to try reading spreadsheet from where you saved it in a data directory 
df = pd.read_excel('43017_2021_246_MOESM1_ESM.xlsx', 'Greenland', usecols=use_cols, skiprows = lambda x: x not in import_rows)

# View pandas dataset head (first 5 rows)
df.head()

In [ ]:
# Create GeoPandas GeoDataFrame which offers additional functionality beyond pandas 
# of a geometry column of Shapely objects, here Shapely points
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df['Lon. oE'], df['Lat. oN']))

# Set the Coordinate Reference System (CRS) of the geodataframe
if gdf.crs is None: 
    # set CRS WGS84 in lon, lat
    gdf.set_crs('epsg:4326', inplace=True)
    
# Display GeoDataFrame
gdf

In [ ]:
# Let's look Greenland's active subglacial lake inventory
gdf[gdf['Lake Type'] == 'Active']

In [ ]:
# Load "Natural Earth” countries dataset, bundled with GeoPandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Create a rough plot to ensure data was read properly
fig, ax = plt.subplots()
world[world['name'] == 'Greenland'].plot(ax=ax, facecolor='lightgray', edgecolor='gray')
gdf[gdf['Lake Type'] == 'Active'].plot(ax=ax, label='active subglacial lake')
ax.set_xlabel('longitude'); ax.set_ylabel('latitude')
ax.set_title('Active subglacial lake distribution \nacross Greenland')
plt.legend()
plt.show()

In [ ]:
# Load "Natural Earth” countries dataset, bundled with GeoPandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
Greenland = world[world['name'] == 'Greenland']
type(Greenland)

In [ ]:
# Create GeoSeries of our GeoDataFrame that converts to Arcic polar stereographic projection 
# and makes 5-km radius buffered polygon around each lake point
gs = gdf.to_crs('3995').buffer(5000)

# Create new GeoDataFrame to store the polygons
gdf_polys = gpd.GeoDataFrame(gdf, geometry=gs, crs='epsg:3995').to_crs('4623')

# Look at active lakes to ensure it worked as expected
gdf_polys[gdf_polys['Lake Type'] == 'Active']

In [ ]:
# export to geopackage to subset ICESat-2 data
gdf_polys[gdf_polys['Name / Location'] == 'Flade Isblink ice cap'].to_file(os.getcwd() + '/Flade_Isblink_poly.gpkg')

## Data
- We will be working with cloud-hosted data files. This [guide](https://nsidc.org/data/user-resources/help-center/nasa-earthdata-cloud-data-access-guide) explains how to find and access Earthdata cloud-hosted data. And [here](https://nsidc.org/data/earthdata-cloud) is a complete list of earthdata cloud-hosted data products available from NSIDC.
- To read cloud-hosted data from NSIDC we must get temporary AWS S3 credentials: “accessKeyId”, “secretAccessKey”, and “sessionToken” [here](https://data.nsidc.earthdatacloud.nasa.gov/s3credentials). 
- You must be authenticated into NASA Earthdata ([click here](https://urs.earthdata.nasa.gov/home) to register if you need an Earthdata account).

In [ ]:
# Specifying the necessary icepyx parameters
short_name = 'ATL15'
spatial_extent = 'Flade_Isblink_poly.gpkg' 
date_range = ['2018-09-15','2023-03-02']

In [ ]:
# Setup the Query object
region = ipx.Query(short_name, spatial_extent, date_range)

In [ ]:
# Visualize area of interest
region.visualize_spatial_extent()

In [ ]:
gran_ids = region.avail_granules()
gran_ids

In [ ]:
gran_ids = region.avail_granules(ids=True, cloud=True)
gran_ids

In [ ]:
s3url = gran_ids[1][0]
s3url

In [ ]:
# icepyx doesn't have the s3 urls for the ATL15 data product yet,
# so we pulled these from NASA Earth Data
# Learn more here: https://nsidc.org/data/user-resources/help-center/nasa-earthdata-cloud-data-access-guide
s3url = 's3://nsidc-cumulus-prod-protected/ATLAS/ATL15/002/2019/ATL15_GL_0314_01km_002_01.nc'

In [ ]:
# Configure these two according to your credentials
earthdata_uid = 'icepyx_devteam'
earthdata_email = 'icepyx.dev@gmail.com'

region.earthdata_login(earthdata_uid, earthdata_email, s3token=True)

In [ ]:
credentials = region._s3login_credentials

In [ ]:
s3 = s3fs.S3FileSystem(key=credentials['accessKeyId'],
                       secret=credentials['secretAccessKey'],
                       token=credentials['sessionToken'])

In [ ]:
# Open the file
%time f = h5py.File(s3.open(s3url,'rb'),'r')

In [ ]:
# View the data file's dictionary keys
list(f.keys())

In [ ]:
# Open s3url data file and store in Xarry Dataset
with s3.open(s3url,'rb') as f:
    # ATL15_dh = rioxarray.open_rasterio(f.read(), group='delta_h')  # FIXME: preferred, but giving error
    ATL15_dh = xr.open_dataset(f, group='delta_h')

# View Xarray Dataset
ATL15_dh

In [ ]:
gdf[gdf['Lake Type'] == 'Active']

In [ ]:
# Let's isolate one of the active subglacial lakes into a variable
flade_isblink = gdf_polys[gdf_polys['Name / Location'] == 'Flade Isblink ice cap']
flade_isblink = world[world['name'] == 'Greenland']

# We can look at the lon, lat (x, y) extrema of the polygon with the .bounds method
flade_isblink.bounds

In [ ]:
# define bounding box extrema based on lake polygons we created earlier
xmin = flade_isblink.bounds.minx.values[0]
ymin = flade_isblink.bounds.minx.values[0]
xmax = flade_isblink.bounds.minx.values[0]
ymax = flade_isblink.bounds.minx.values[0]

# Clip dataset to region of interest for faster plotting
mask_x = (ATL15_dh.x >= xmin) & (ATL15_dh.x <= xmax)
mask_y = (ATL15_dh.y >= ymin) & (ATL15_dh.y <= ymax)
ATL15_dh_sub = ATL15_dh.where(mask_x & mask_y, drop=True)

In [ ]:
ATL15_dh

In [ ]:
type(ATL15_dh)

In [ ]:
fig, ax = plt.subplots()
world[world['name'] == 'Greenland'].to_crs(3995).boundary.plot(ax=ax)
dhdt = ATL15_dh['delta_h'][0,:,:] - ATL15_dh['delta_h'][0,:,:]
ax.imshow(dhdt, extent=[x_min, x_max, y_min, y_max])

In [ ]:
# clean up environment by deleting intermediary files
os.remove('Flade_Isblink_poly.gpkg')
os.remove('core.257')

In [ ]:
### My Subsection


In [ ]:
## Summary


## References

Livingstone, S.J., Li, Y., Rutishauser, A. et al. Subglacial lakes and their changing role in a warming climate. Nat Rev Earth Environ 3, 106–124 (2022). https://doi.org/10.1038/s43017-021-00246-9